In [1]:
import numpy as np
import pandas as pd
import scipy
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

In [3]:
import yfinance as yf

raw_data = yf.download(
    tickers=["^GSPC", "^FTSE", "^N225", "^GDAXI"], 
    start="1994-01-07", 
    end="2018-01-29", 
    interval="1d", 
    group_by="ticker", 
    auto_adjust=True, 
    threads=True
)

print(raw_data)

[*********************100%***********************]  4 of 4 completed

Ticker            ^GDAXI                                            \
Price               Open          High           Low         Close   
Date                                                                 
1994-01-07   2218.959961   2227.639893   2201.820068   2224.949951   
1994-01-10   2231.840088   2238.010010   2222.000000   2225.000000   
1994-01-11   2225.429932   2235.610107   2225.179932   2228.100098   
1994-01-12   2227.120117   2227.790039   2182.060059   2182.060059   
1994-01-13   2171.500000   2183.709961   2134.100098   2142.370117   
...                  ...           ...           ...           ...   
2018-01-22  13435.139648  13470.049805  13410.959961  13463.690430   
2018-01-23  13577.139648  13596.889648  13517.809570  13559.599609   
2018-01-24  13572.709961  13576.839844  13414.740234  13414.740234   
2018-01-25  13374.379883  13443.360352  13222.469727  13298.360352   
2018-01-26  13297.509766  13348.690430  13268.309570  13340.169922   

Ticker             

In [5]:
df_comp = raw_data.copy()

In [8]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [9]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp = df_comp.asfreq('B')
df_comp = df_comp.fillna(method='ffill')

In [10]:
df_comp['ret_spx'] = df_comp['spx'].pct_change(1) * 100
df_comp['ret_ftse'] = df_comp['ftse'].pct_change(1) * 100
df_comp['ret_dax'] = df_comp['dax'].pct_change(1) * 100
df_comp['ret_nikkei'] = df_comp['nikkei'].pct_change(1) * 100

In [11]:
size = int(len(df_comp) * 0.8)
df_train, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [12]:
from pmdarima import auto_arima

In [ ]:
model_auto = auto_arima(df_comp['ret_ftse'][1:]) 


In [11]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [12]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 6274
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -9551.977
Date:                Fri, 13 Dec 2024   AIC                          19125.954
Time:                        00:04:35   BIC                          19200.140
Sample:                    01-11-1994   HQIC                         19151.658
                         - 01-26-2018                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0271      0.018      1.490      0.136      -0.009       0.063
ar.L1          0.0806      0.081      0.998      0.318      -0.078       0.239
ar.L2         -0.5808      0.080     -7.295      0.000      -0.737      -0.425
ar.L3         -0.1271      0.072     -1.761      0.078      -0.269       0.014
ar.L4          0.2579      0.076      3.405      0.001       0.109       0.406
ma.L1         -0.1001      0.081     -1.239      0.215      -0.258       0.058
ma.L2          0.5345      0.081      6.594      0.000       0.376       0.693
ma.L3          0.0495      0.071      0.699      0.484      -0.089       0.188
ma.L4         -0.2567      0.076     -3.394      0.001      -0.405      -0.108
ma.L5         -0.0894      0.009    -10.439      0.000      -0.106      -0.073
sigma2         1.2307      0.012    103.973      0.000       1.208       1.254
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8790.78
Prob(Q):                              0.97   Prob(JB):                         0.00
Heteroskedasticity (H):               0.87   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [13]:
model_auto = auto_arima(
    df_comp['ret_ftse'][1:], 
    exogenous=df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], 
    m=5, 
    max_order=None, 
    max_p=7, 
    max_q=7, 
    max_d=2, 
    max_P=4, 
    max_Q=4, 
    max_D=2,
    maxiter=50, 
    alpha=0.05, 
    n_jobs=-1, 
    trend='ct', 
    information_criterion='oob', 
    out_of_sample_size=int(len(df_comp) * 0.2)
)



In [15]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 6274
Model:             SARIMAX(0, 0, 3)x(2, 0, [1, 2], 5)   Log Likelihood               -9581.139
Date:                                Fri, 13 Dec 2024   AIC                          19182.278
Time:                                        00:21:49   BIC                          19249.719
Sample:                                             0   HQIC                         19205.645
                                               - 6274                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0283      0.039      0.716      0.474      -0.049       0.106
drift      -2.361e-06   1.06e-05     -0.223      0.824   -2.31e-05    1.84e-05
ma.L1         -0.0242      0.009     -2.752      0.006      -0.041      -0.007
ma.L2         -0.0503      0.008     -6.351      0.000      -0.066      -0.035
ma.L3         -0.0840      0.008    -10.746      0.000      -0.099      -0.069
ar.S.L5       -0.0949      0.724     -0.131      0.896      -1.514       1.324
ar.S.L10      -0.1821      0.203     -0.899      0.369      -0.579       0.215
ma.S.L5        0.0420      0.724      0.058      0.954      -1.377       1.461
ma.S.L10       0.1657      0.231      0.717      0.473      -0.287       0.619
sigma2         1.3682      0.014     95.553      0.000       1.340       1.396
===================================================================================
Ljung-Box (L1) (Q):                   0.14   Jarque-Bera (JB):              9004.63
Prob(Q):                              0.71   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                            -0.23
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.85
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""